Depedencies installation

In [1]:
!pip install pymupdf
!pip install pdfminer.six
!pip install KeyBERT
!pip install yake
!pip install pdfplumber

  Using cached PyMuPDF-1.22.3-cp310-cp310-win_amd64.whl (11.7 MB)
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)
  Using cached keybert-0.7.0-py3-none-any.whl
     -------------------------------------- 239.4/239.4 kB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 87.5/87.5 kB 4.8 MB/s eta 0:00:00
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Using cached yake-0.4.8-py2.py3-none-any.whl (60 kB)
  Using cached segtok-1.5.11-py3-none-any.whl (24 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached jellyfish-0.11.2-cp310-none-win_amd64.whl (208 kB)
     ---------------------------------------- 46.1/46.1 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 143.6/143.6 kB 8.9 MB/s eta 0:00:00


Import Libraries

In [2]:
## Import PDF Reader PyMuPDF
import fitz

## Import PDF Reader PDFMiner.six
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from io import StringIO

## Import PDFPlumber
import pdfplumber

## Import widget
from IPython.display import Image as IPImage, display, HTML
from ipywidgets import interact, Select, SelectMultiple, HBox, VBox, interactive_output, widgets, Layout
import warnings

## Import Keyword Extractor KeyBERT
from keybert import KeyBERT
import yake

## Other
import glob, os, sys, re

In [9]:
pdf_path = "../paper/"
pdf_reader_list = ['PDFMiner.six', 'PyMuPDF', 'PDFPlumber']
opt = ['text', 'metadata']

In [4]:
## Extract options
page_number = 1


Text Extraction

In [5]:
def pdfminer_extract_text(file_path, page_number):
  """

  """
  resource_manager = PDFResourceManager()
  output_string = StringIO()
  codec = 'utf-8'
  laparams = LAParams()

  with open(file_path, 'rb') as file:
      interpreter = PDFPageInterpreter(resource_manager, TextConverter(resource_manager, output_string, codec=codec, laparams=laparams))
      page = list(PDFPage.get_pages(file))[page_number - 1]  # Subtract 1 to account for 0-based indexing
      interpreter.process_page(page)

  text = output_string.getvalue()
  output_string.close()

  return text

def pymupdf_extract_text(file_path, page_number):
  """
  
  """
  pdf = fitz.open(file_path)
  text = pdf.load_page(page_number - 1).get_text()
  pdf.close()
  return text

def pdfplumber_extract_text(file_path, page_number):
  """
  """
  pdf = pdfplumber.open(file_path)
  text = pdf.pages[page_number - 1].extract_text(x_tolerance=3, y_tolerance=3, layout=False, x_density=1.25, y_density=13)
  pdf.close()
  return text

def pdf_extract_text(pdf_reader, file_path, page_number):
  text = ""
  if pdf_reader == 'PDFMiner.six':
    text = pdfminer_extract_text(file_path, page_number)
  elif pdf_reader == 'PyMuPDF':
    text =pymupdf_extract_text(file_path, page_number)
  elif pdf_reader == 'PDFPlumber':
    text =pdfplumber_extract_text(file_path, page_number)
  return text

Metadata Extraction

In [6]:
def pdfminer_metadata(file_path):
  with open(file_path, 'rb') as file:
    # Create a PDF parser object
    parser = PDFParser(file)
    # Create a PDF document object
    document = PDFDocument(parser)

    metadata = ""
    # Check if the document has metadata
    if hasattr(document, 'info') and document.info is not None:
        # Access the metadata dictionary
        metadata = document.info
    else:
        print('No metadata found in the PDF.')
  return metadata

def pymupdf_metadata(file_path):
  pdf = fitz.open(file_path)
  metadata = pdf.metadata
  pdf.close()
  return metadata

def pdfplumber_metadata(file_path):
  pdf = pdfplumber.open(file_path)
  metadata = pdf.metadata
  pdf.close()
  return metadata

def pdf_metadata(pdf_reader, file_path):
  metadata = ""
  if pdf_reader == 'PDFMiner.six':
    metadata = pdfminer_metadata(file_path)
  elif pdf_reader == 'PyMuPDF':
    metadata = pymupdf_metadata(file_path)
  elif pdf_reader == 'PDFPlumber':
    metadata = pdfplumber_metadata(file_path)
  return metadata

In [7]:
def fine_replace(text):
  text = text.replace("-\n", "")
  text = text.replace("\n", " ")
  text = text.replace("'", "")
  return text

In [10]:
def read_file(pdf_file, pdf_reader, opt, fr, pg):
    global data_text
    path = pdf_path + pdf_file
    if opt == 'text':
        data_text = pdf_extract_text(pdf_reader, path, pg)
        if fr == 'Yes':
          data_text = fine_replace(data_text)
    elif opt == 'metadata':
        data_text = pdf_metadata(pdf_reader, path)
    display(widgets.HTML(data_text, raw=True))

pdf_list = [os.path.basename(file) for file in glob.iglob( pdf_path +'/*') if re.search(".+\.(pdf)", os.path.basename(file))]
fr = ['Yes', 'No']
pg = [1,2,3,4,5,6,7]

w1 = widgets.Select(options=pdf_list, value=pdf_list[0],rows=3, description='PDF File',disabled=False, layout=Layout(width='25%'))
w2 = widgets.Select(options=pdf_reader_list, value=pdf_reader_list[0],rows=3, description='PDF Reader',disabled=False, layout=Layout(width='20%'))
w3 = widgets.Select(options=opt, value=opt[0],rows=3, description='Options',disabled=False, layout=Layout(width='15%'))
w4 = widgets.Select(options=fr, value=fr[1],rows=3, description='\nReplace',disabled=False, layout=Layout(width='10%'))
w5 = widgets.Select(options=pg, value=pg[0],rows=3, description='Page',disabled=False, layout=Layout(width='10%'))
ui = HBox([w1, w2, w3, w4, w5])
w_dict = {'pdf_file': w1, 'pdf_reader': w2, 'opt': w3, 'fr': w4, 'pg': w5}
out = interactive_output(read_file, w_dict)
display(ui, out)

Output()

In [11]:
data_text

'Design and Control of Roller Grasper V2 for In-Hand Manipulation\n\nShenli Yuan1,2, Lin Shao2, Connor L. Yako1,2, Alex Gruebele1, and J. Kenneth Salisbury2\n\n0\n2\n0\n2\n\nv\no\nN\n7\n1\n\n]\n\nO\nR\n.\ns\nc\n[\n\n2\nv\n9\n9\n4\n8\n0\n.\n4\n0\n0\n2\n:\nv\ni\nX\nr\na\n\nAbstract— The ability to perform in-hand manipulation\nstill remains an unsolved problem; having this capability\nwould allow robots to perform sophisticated tasks requiring\nrepositioning and reorienting of grasped objects. In this work,\nwe present a novel non-anthropomorphic robot grasper with\nthe ability to manipulate objects by means of active surfaces at\nthe ﬁngertips. Active surfaces are achieved by spherical rolling\nﬁngertips with two degrees of freedom (DoF) – a pivoting\nmotion for surface reorientation – and a continuous rolling\nmotion for moving the object. A further DoF is in the base\nof each ﬁnger, allowing the ﬁngers to grasp objects over a\nrange of size and shapes. Instantaneous kinematics was der

In [12]:
len(data_text.split())

748

Keyword Extraction

In [13]:
# Create candidates
kw_extractor = yake.KeywordExtractor(top=100)
candidates = kw_extractor.extract_keywords(data_text)
candidates = [candidate[0] for candidate in candidates]

i = 0
for candidate in candidates:
  candidates[i] = candidate.lower()
  i = i+ 1

candidates

['in-hand manipulation shenli',
 'in-hand manipulation',
 'perform in-hand manipulation',
 'manipulation shenli',
 'manipulation',
 'object',
 'in-hand',
 'perform in-hand',
 'objects',
 'grasper',
 'lin',
 'shenli',
 'connor',
 'perform',
 'roller',
 'in-hand manipulation tasks',
 'roller grasper',
 'stanford university',
 'manipulation tasks',
 'learning',
 'degrees of freedom',
 'active surfaces',
 'tasks',
 'sophisticated object manipulation',
 'grasp',
 'rolling',
 'robots',
 'robot',
 'object manipulation',
 'control',
 'spherical rollers',
 'rollers',
 'stanford',
 'active',
 'ﬁnger',
 'motion',
 'degrees',
 'freedom',
 'contact',
 'spherical',
 'roller orientations',
 'design',
 'in-hand manipulation literature',
 'surfaces',
 'ability',
 'ﬁngertips',
 'grasp objects',
 'robot grasper',
 'artiﬁcial intelligence lab',
 'graspers',
 'rolling contact',
 'stanford interdisciplinary',
 'fig.',
 'university',
 'hands',
 'alex',
 'kenneth',
 'learn',
 'control scheme',
 'non-anthropom

In [15]:
# Pass candidates to KeyBERT
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(data_text, keyphrase_ngram_range=(1, 10), candidates=candidates, stop_words='english', use_mmr=True, diversity=0.4)
keywords

[('robot grasper', 0.6976),
 ('roller grasper', 0.697),
 ('learn dexterous manipulation', 0.5413),
 ('spherical rolling ﬁngertips', 0.4327),
 ('active surface achieved', 0.3491)]